In [37]:
# Run this cell without modification

# Import some packages that we need to use
from utils import *
import gym
import numpy as np
from collections import deque

In [38]:
# Solve the TODOs and remove `pass`

def _render_helper(env):
    env.render()
    wait(sleep=0.2)


def evaluate(policy, num_episodes, seed=0, env_name='FrozenLake8x8-v1', render=False):
    """[TODO] You need to implement this function by yourself. It
    evaluate the given policy and return the mean episode reward.
    We use `seed` argument for testing purpose.
    You should pass the tests in the next cell.

    :param policy: a function whose input is an interger (observation)
    :param num_episodes: number of episodes you wish to run
    :param seed: an interger, used for testing.
    :param env_name: the name of the environment
    :param render: a boolean flag. If true, please call _render_helper
    function.
    :return: the averaged episode reward of the given policy.
    """

    # Create environment (according to env_name, we will use env other than 'FrozenLake8x8-v0')
    env = gym.make(env_name)

    # Seed the environment
    env.seed(seed)

    # Build inner loop to run.
    # For each episode, do not set the limit.
    # Only terminate episode (reset environment) when done = True.
    # The episode reward is the sum of all rewards happen within one episode.
    # Call the helper function `render(env)` to render
    rewards = []
    for i in range(num_episodes):
        # reset the environment
        obs = env.reset()
        act = policy(obs)
        
        ep_reward = 0
        while True:
            # [TODO] run the environment and terminate it if done, collect the
            # reward at each step and sum them to the episode reward.
            if render:
                _render_helper(env)
            obs, reward, done, info = env.step(policy(obs))
            ep_reward += reward
            if done:
                break   
        rewards.append(ep_reward)

    return np.mean(rewards)

# [TODO] Run next cell to test your implementation!

In [39]:
# Run this cell without modification

class TabularRLTrainerAbstract:
    """This is the abstract class for tabular RL trainer. We will inherent the specify 
    algorithm's trainer from this abstract class, so that we can reuse the codes like
    getting the dynamic of the environment (self._get_transitions()) or rendering the
    learned policy (self.render())."""
    
    def __init__(self, env_name='FrozenLake8x8-v1', model_based=True):
        self.env_name = env_name
        self.env = gym.make(self.env_name)
        self.action_dim = self.env.action_space.n
        self.obs_dim = self.env.observation_space.n
        
        self.model_based = model_based

    def _get_transitions(self, state, act):
        """Query the environment to get the transition probability,
        reward, the next state, and done given a pair of state and action.
        We implement this function for you. But you need to know the 
        return format of this function.
        """
        self._check_env_name()
        assert self.model_based, "You should not use _get_transitions in " \
            "model-free algorithm!"
        
        # call the internal attribute of the environments.
        # `transitions` is a list contain all possible next states and the 
        # probability, reward, and termination indicater corresponding to it
        transitions = self.env.env.P[state][act]

        # Given a certain state and action pair, it is possible
        # to find there exist multiple transitions, since the 
        # environment is not deterministic.
        # You need to know the return format of this function: a list of dicts
        ret = []
        for prob, next_state, reward, done in transitions:
            ret.append({
                "prob": prob,
                "next_state": next_state,
                "reward": reward,
                "done": done
            })
        return ret
    
    def _check_env_name(self):
        assert self.env_name.startswith('FrozenLake')

    def print_table(self):
        """print beautiful table, only work for FrozenLake8X8-v1 env. We 
        write this function for you."""
        self._check_env_name()
        print_table(self.table)

    def train(self):
        """Conduct one iteration of learning."""
        raise NotImplementedError("You need to override the "
                                  "Trainer.train() function.")

    def evaluate(self):
        """Use the function you write to evaluate current policy.
        Return the mean episode reward of 1000 episodes when seed=0."""
        result = evaluate(self.policy, 1000, env_name=self.env_name)
        return result

    def render(self):
        """Reuse your evaluate function, render current policy 
        for one episode when seed=0"""
        evaluate(self.policy, 1, render=True, env_name=self.env_name)

In [40]:
# Solve the TODOs and remove `pass`

class SARSATrainer(TabularRLTrainerAbstract):
    def __init__(self,
                 gamma=1.0,
                 eps=0.1,
                 learning_rate=1.0,
                 max_episode_length=100,
                 env_name='FrozenLake8x8-v1'
                 ):
        super(SARSATrainer, self).__init__(env_name, model_based=False)

        # discount factor
        self.gamma = gamma

        # epsilon-greedy exploration policy parameter
        self.eps = eps

        # maximum steps in single episode
        self.max_episode_length = max_episode_length

        # the learning rate
        self.learning_rate = learning_rate

        # build the Q table
        # [TODO] uncomment the next line, pay attention to the shape
        self.table = np.zeros((self.obs_dim, self.action_dim))

    def policy(self, obs):
        """Implement epsilon-greedy policy

        It is a function that take an integer (state / observation)
        as input and return an interger (action).
        """

        # [TODO] You need to implement the epsilon-greedy policy here.
        # hint: We have self.eps probability to choose a unifomly random
        #  action in range [0, 1, .., self.action_dim - 1], 
        #  otherwise choose action that maximize the Q value
        if np.random.random()<self.eps:
            return np.random.choice(4)
        else:
            return np.argmax(self.table[obs])

    def train(self):
        """Conduct one iteration of learning."""
        # [TODO] Q table may be need to be reset to zeros.
        # if you think it should, than do it. If not, then move on.
        
        # No, we should do nothing.

        obs = self.env.reset()
        for t in range(self.max_episode_length):
            act = self.policy(obs)

            next_obs, reward, done, _ = self.env.step(act)
            next_act = self.policy(next_obs)

            # [TODO] compute the TD error, based on the next observation and
            #  action.
            td_error = reward+self.gamma*self.table[next_obs][next_act]-self.table[obs][act]
            
            # [TODO] compute the new Q value
            # hint: use TD error, self.learning_rate and old Q value
            new_value = self.table[obs][act]+self.learning_rate*td_error
            
            self.table[obs][act] = new_value

            # [TODO] Implement (1) break if done. (2) update obs for next 
            #  self.policy(obs) call
            if done:
                break
            else:
                obs=next_obs             

# [TODO] run the next cell to check your code

In [41]:
# Run this cell without modification

# set eps = 0 to disable exploration.
test_trainer = SARSATrainer(eps=0.0)
test_trainer.table.fill(0)

# set the Q value of (obs 0, act 3) to 100, so that it should be taken by 
# policy.
test_obs = 0
test_act = test_trainer.action_dim - 1
test_trainer.table[test_obs][test_act] = 100

# assertion
assert test_trainer.policy(test_obs) == test_act, \
    "Your action is wrong! Should be {} but get {}.".format(
        test_act, test_trainer.policy(test_obs))

# delete trainer
del test_trainer

# set eps = 0 to disable exploitation.
test_trainer = SARSATrainer(eps=1.0)
test_trainer.table.fill(0)

act_set = set()
for i in range(100):
    act_set.add(test_trainer.policy(0))

# assertion
assert len(act_set) > 1, ("You sure your uniformaly action selection mechanism"
                          " is working? You only take action {} when "
                          "observation is 0, though we run trainer.policy() "
                          "for 100 times.".format(act_set))
# delete trainer
del test_trainer

print("Policy Test passed!")

Policy Test passed!


In [42]:
# Solve TODO

# Managing configurations of your experiments is important for your research.
default_sarsa_config = dict(
    max_iteration=20000,
    max_episode_length=200,
    learning_rate=0.01,
    evaluate_interval=1000,
    gamma=0.8,
    eps=0.3,
    env_name='FrozenLakeNotSlippery-v0'
)


def sarsa(train_config=None):
    config = default_sarsa_config.copy()
    if train_config is not None:
        config.update(train_config)

    trainer = SARSATrainer(
        gamma=config['gamma'],
        eps=config['eps'],
        learning_rate=config['learning_rate'],
        max_episode_length=config['max_episode_length'],
        env_name=config['env_name']
    )

    for i in range(config['max_iteration']):
        # train the agent
        trainer.train()  # [TODO] please uncomment this line

        # evaluate the result
        if i % config['evaluate_interval'] == 0:
            print(
                "[INFO]\tIn {} iteration, current mean episode reward is {}."
                "".format(i, trainer.evaluate()))

    if trainer.evaluate() < 0.6:
        print("We expect to get the mean episode reward greater than 0.6. " \
        "But you get: {}. Please check your codes.".format(trainer.evaluate()))

    return trainer

In [43]:
# Run this cell without modification

sarsa_trainer = sarsa()

[INFO]	In 0 iteration, current mean episode reward is 0.0.
[INFO]	In 1000 iteration, current mean episode reward is 0.634.
[INFO]	In 2000 iteration, current mean episode reward is 0.641.
[INFO]	In 3000 iteration, current mean episode reward is 0.672.
[INFO]	In 4000 iteration, current mean episode reward is 0.665.
[INFO]	In 5000 iteration, current mean episode reward is 0.647.
[INFO]	In 6000 iteration, current mean episode reward is 0.65.
[INFO]	In 7000 iteration, current mean episode reward is 0.647.
[INFO]	In 8000 iteration, current mean episode reward is 0.655.
[INFO]	In 9000 iteration, current mean episode reward is 0.685.
[INFO]	In 10000 iteration, current mean episode reward is 0.642.
[INFO]	In 11000 iteration, current mean episode reward is 0.668.
[INFO]	In 12000 iteration, current mean episode reward is 0.637.
[INFO]	In 13000 iteration, current mean episode reward is 0.629.
[INFO]	In 14000 iteration, current mean episode reward is 0.675.
[INFO]	In 15000 iteration, current mean e

In [44]:
# Run this cell without modification

sarsa_trainer.print_table()


=== The state value for action 0 ===
+-----+-----+-----+-----+-----+
|     |   0 |   1 |   2 |   3 |
|-----+-----+-----+-----+-----+
| 0   |0.124|0.122|0.059|0.001|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 1   |0.162|0.000|0.000|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 2   |0.235|0.245|0.326|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 3   |0.000|0.000|0.497|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+


=== The state value for action 1 ===
+-----+-----+-----+-----+-----+
|     |   0 |   1 |   2 |   3 |
|-----+-----+-----+-----+-----+
| 0   |0.164|0.000|0.039|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 1   |0.236|0.000|0.431|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 2   |0.000|0.484|0.734|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 3   |0.000|0.497|0.703|0.000|
|     |     |     |     |  

In [45]:
# Run this cell without modification

sarsa_trainer.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


In [46]:
# Solve the TODOs and remove `pass`

class QLearningTrainer(TabularRLTrainerAbstract):
    def __init__(self,
                 gamma=1.0,
                 eps=0.1,
                 learning_rate=1.0,
                 max_episode_length=100,
                 env_name='FrozenLake8x8-v1'
                 ):
        super(QLearningTrainer, self).__init__(env_name, model_based=False)
        self.gamma = gamma
        self.eps = eps
        self.max_episode_length = max_episode_length
        self.learning_rate = learning_rate

        # build the Q table
        self.table = np.zeros((self.obs_dim, self.action_dim))

    def policy(self, obs):
        """Implement epsilon-greedy policy

        It is a function that take an integer (state / observation)
        as input and return an interger (action).
        """

        # [TODO] You need to implement the epsilon-greedy policy here.
        # hint: Just copy your codes in SARSATrainer.policy()
        if np.random.random()<self.eps:
            return np.random.choice(4)
        else:
            return np.argmax(self.table[obs])

    def train(self):
        """Conduct one iteration of learning."""
        # [TODO] Q table may be need to be reset to zeros.
        # if you think it should, than do it. If not, then move on.
        pass
        # No, we should do nothing.

        obs = self.env.reset()
        for t in range(self.max_episode_length):
            act = self.policy(obs)

            next_obs, reward, done, _ = self.env.step(act)

            # [TODO] compute the TD error, based on the next observation
            # hint: we do not need next_act anymore.
            td_error = reward+self.gamma*np.max(self.table[next_obs])-self.table[obs][act]
            
            # [TODO] compute the new Q value
            # hint: use TD error, self.learning_rate and old Q value
            new_value = self.table[obs][act]+self.learning_rate*td_error
            
            self.table[obs][act] = new_value
            obs = next_obs
            if done:
                break

In [47]:
# Solve the TODO

# Managing configurations of your experiments is important for your research.
default_q_learning_config = dict(
    max_iteration=20000,
    max_episode_length=200,
    learning_rate=0.01,
    evaluate_interval=1000,
    gamma=0.8,
    eps=0.3,
    env_name='FrozenLakeNotSlippery-v0'
)


def q_learning(train_config=None):
    config = default_q_learning_config.copy()
    if train_config is not None:
        config.update(train_config)

    trainer = QLearningTrainer(
        gamma=config['gamma'],
        eps=config['eps'],
        learning_rate=config['learning_rate'],
        max_episode_length=config['max_episode_length'],
        env_name=config['env_name']
    )

    for i in range(config['max_iteration']):
        # train the agent
        trainer.train()  # [TODO] please uncomment this line

        # evaluate the result
        if i % config['evaluate_interval'] == 0:
            print(
                "[INFO]\tIn {} iteration, current mean episode reward is {}."
                "".format(i, trainer.evaluate()))

    if trainer.evaluate() < 0.6:
        print("We expect to get the mean episode reward greater than 0.6. " \
        "But you get: {}. Please check your codes.".format(trainer.evaluate()))

    return trainer

In [48]:
# Run this cell without modification

q_learning_trainer = q_learning()

[INFO]	In 0 iteration, current mean episode reward is 0.0.
[INFO]	In 1000 iteration, current mean episode reward is 0.0.
[INFO]	In 2000 iteration, current mean episode reward is 0.0.
[INFO]	In 3000 iteration, current mean episode reward is 0.0.
[INFO]	In 4000 iteration, current mean episode reward is 0.667.
[INFO]	In 5000 iteration, current mean episode reward is 0.661.
[INFO]	In 6000 iteration, current mean episode reward is 0.627.
[INFO]	In 7000 iteration, current mean episode reward is 0.662.
[INFO]	In 8000 iteration, current mean episode reward is 0.641.
[INFO]	In 9000 iteration, current mean episode reward is 0.652.
[INFO]	In 10000 iteration, current mean episode reward is 0.667.
[INFO]	In 11000 iteration, current mean episode reward is 0.66.
[INFO]	In 12000 iteration, current mean episode reward is 0.647.
[INFO]	In 13000 iteration, current mean episode reward is 0.643.
[INFO]	In 14000 iteration, current mean episode reward is 0.667.
[INFO]	In 15000 iteration, current mean episode

In [49]:
# Run this cell without modification

q_learning_trainer.print_table()


=== The state value for action 0 ===
+-----+-----+-----+-----+-----+
|     |   0 |   1 |   2 |   3 |
|-----+-----+-----+-----+-----+
| 0   |0.262|0.262|0.026|0.026|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 1   |0.328|0.000|0.000|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 2   |0.410|0.410|0.512|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 3   |0.000|0.000|0.640|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+


=== The state value for action 1 ===
+-----+-----+-----+-----+-----+
|     |   0 |   1 |   2 |   3 |
|-----+-----+-----+-----+-----+
| 0   |0.328|0.000|0.416|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 1   |0.410|0.000|0.640|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 2   |0.000|0.640|0.800|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 3   |0.000|0.532|0.800|0.000|
|     |     |     |     |  

In [50]:
# Run this cell without modification

q_learning_trainer.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


In [51]:
# Solve the TODOs and remove `pass`

class MCControlTrainer(TabularRLTrainerAbstract):
    def __init__(self,
                 gamma=1.0,
                 eps=0.3,
                 max_episode_length=100,
                 env_name='FrozenLake8x8-v1'
                 ):
        super(MCControlTrainer, self).__init__(env_name, model_based=False)
        self.gamma = gamma
        self.eps = eps
        self.max_episode_length = max_episode_length

        # build the dict of lists
        self.returns = {}
        for obs in range(self.obs_dim):
            for act in range(self.action_dim):
                self.returns[(obs, act)] = []

        # build the Q table
        self.table = np.zeros((self.obs_dim, self.action_dim))

    def policy(self, obs):
        """Implement epsilon-greedy policy

        It is a function that take an integer (state / observation)
        as input and return an interger (action).
        """

        # [TODO] You need to implement the epsilon-greedy policy here.
        # hint: Just copy your codes in SARSATrainer.policy()
        if np.random.random()<self.eps:
            return np.random.choice(4)
        else:
            return np.argmax(self.table[obs])
            
    def train(self):
        """Conduct one iteration of learning."""
        observations = []
        actions = []
        rewards = []

        # [TODO] rollout for one episode, store data in three lists create 
        #  above.
        # hint: we do not need to store next observation.
        obs = self.env.reset()
        for _ in range(self.max_episode_length):
            act = self.policy(obs)
            next_obs, reward, done, _ = self.env.step(act)
            observations.append(obs)
            actions.append(act)
            rewards.append(reward)
            if done:
                break
            else:
                obs = next_obs                

        assert len(actions) == len(observations)
        assert len(actions) == len(rewards)

        occured_state_action_pair = set()
        length = len(actions)
        value = 0
        for i in reversed(range(length)):
            # if length = 10, then i = 9, 8, ..., 0

            obs = observations[i]
            act = actions[i]
            reward = rewards[i]

            # [TODO] compute the value reversely
            # hint: value(t) = gamma * value(t+1) + r(t)
            value = self.gamma*value+reward

            if (obs, act) not in occured_state_action_pair:
                occured_state_action_pair.add((obs, act))

                # [TODO] append current return (value) to dict
                # hint: `value` represents the future return due to 
                #  current (obs, act), so we need to store this value
                #  in trainer.returns
                self.returns[(obs, act)].append(value)

                # [TODO] compute the Q value from self.returns and write it 
                #  into self.table
                self.table[obs][act] = np.mean(self.returns[(obs, act)])

                # we don't need to update the policy since it is 
                # automatically adjusted with self.table

In [52]:
# Run this cell without modification

# Managing configurations of your experiments is important for your research.
default_mc_control_config = dict(
    max_iteration=20000,
    max_episode_length=200,
    evaluate_interval=1000,
    gamma=0.8,
    eps=0.3,
    env_name='FrozenLakeNotSlippery-v0'
)


def mc_control(train_config=None):
    config = default_mc_control_config.copy()
    if train_config is not None:
        config.update(train_config)

    trainer = MCControlTrainer(
        gamma=config['gamma'],
        eps=config['eps'],
        max_episode_length=config['max_episode_length'],
        env_name=config['env_name']
    )

    for i in range(config['max_iteration']):
        # train the agent
        trainer.train()

        # evaluate the result
        if i % config['evaluate_interval'] == 0:
            print(
                "[INFO]\tIn {} iteration, current mean episode reward is {}."
                "".format(i, trainer.evaluate()))

    if trainer.evaluate() < 0.6:
        print("We expect to get the mean episode reward greater than 0.6. " \
        "But you get: {}. Please check your codes.".format(trainer.evaluate()))

    return trainer

In [54]:
# Run this cell without modification

mc_control_trainer = mc_control()

sarsa_trainer = sarsa()

[INFO]	In 0 iteration, current mean episode reward is 0.0.
[INFO]	In 1000 iteration, current mean episode reward is 0.0.
[INFO]	In 2000 iteration, current mean episode reward is 0.0.
[INFO]	In 3000 iteration, current mean episode reward is 0.0.
[INFO]	In 4000 iteration, current mean episode reward is 0.0.
[INFO]	In 5000 iteration, current mean episode reward is 0.0.
[INFO]	In 6000 iteration, current mean episode reward is 0.001.
[INFO]	In 7000 iteration, current mean episode reward is 0.0.
[INFO]	In 8000 iteration, current mean episode reward is 0.0.
[INFO]	In 9000 iteration, current mean episode reward is 0.0.
[INFO]	In 10000 iteration, current mean episode reward is 0.0.
[INFO]	In 11000 iteration, current mean episode reward is 0.623.
[INFO]	In 12000 iteration, current mean episode reward is 0.649.
[INFO]	In 13000 iteration, current mean episode reward is 0.665.
[INFO]	In 14000 iteration, current mean episode reward is 0.65.
[INFO]	In 15000 iteration, current mean episode reward is 0

In [32]:
# Run this cell without modification

mc_control_trainer.print_table()


=== The state value for action 0 ===
+-----+-----+-----+-----+-----+
|     |   0 |   1 |   2 |   3 |
|-----+-----+-----+-----+-----+
| 0   |0.038|0.012|0.042|0.145|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 1   |0.054|0.000|0.000|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 2   |0.114|0.163|0.289|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 3   |0.000|0.000|0.510|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+


=== The state value for action 1 ===
+-----+-----+-----+-----+-----+
|     |   0 |   1 |   2 |   3 |
|-----+-----+-----+-----+-----+
| 0   |0.144|0.000|0.321|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 1   |0.228|0.000|0.504|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 2   |0.000|0.515|0.747|0.000|
|     |     |     |     |     |
+-----+-----+-----+-----+-----+
| 3   |0.000|0.509|0.739|0.000|
|     |     |     |     |  

In [33]:
# Run this cell without modification

mc_control_trainer.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [35]:
# It's ok to leave this cell commented.

new_config = dict(
    env_name="FrozenLake8x8-v1"
)

new_mc_control_trainer = mc_control(new_config)
new_q_learning_trainer = q_learning(new_config)
new_sarsa_trainer = sarsa(new_config)

[INFO]	In 0 iteration, current mean episode reward is 0.0.
[INFO]	In 1000 iteration, current mean episode reward is 0.0.
[INFO]	In 2000 iteration, current mean episode reward is 0.0.
[INFO]	In 3000 iteration, current mean episode reward is 0.0.
[INFO]	In 4000 iteration, current mean episode reward is 0.0.
[INFO]	In 5000 iteration, current mean episode reward is 0.0.


KeyboardInterrupt: 